<a href="https://colab.research.google.com/github/canast07/EPL448-Ticket-price/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Approach 1

In [5]:
import pandas as pd

Removing problematic rows (rows that contain null fields) 

In [48]:
print("Records with nulls: ",trainData.isnull().sum().sum())
trainData = trainData.dropna()
print("Records with nulls after cleanup: ",trainData.isnull().sum().sum())


Records with nulls:  1
Records with nulls after cleanup:  0


Loading data in variable trainData

In [14]:
trainData = pd.read_excel("Data_Train.xlsx")
trainData.pop("Route") # Route is duplicate information because of source-destination
trainData.pop("Source") 
trainData.pop("Destination") # Source - Destination can be replaced with Duration time 

,Airline,Date_of_Journey,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,16:50,21:35,4h 45m,1 stop,No info,13302


1	No info
2	1 Long layover
3	2 Long layover
4	1 Short layover
5	Business class
6	Change airports
7	In-flight meal not included
8	Red-eye flight
9	No check-in baggage included
For column Additional_Info we have those 9 categories. We are going to replace the additional with 9 new columns (1 each category)

In [27]:
trainData= pd.get_dummies(trainData, columns= ['Additional_Info'],drop_first=True)
trainData.head()

,Airline,Date_of_Journey,Dep_Time,Arrival_Time,Duration,Total_Stops,Price,Additional_Info_1 Short layover,Additional_Info_2 Long layover,Additional_Info_Business class,Additional_Info_Change airports,Additional_Info_In-flight meal not included,Additional_Info_No Info,Additional_Info_No check-in baggage included,Additional_Info_No info,Additional_Info_Red-eye flight
0,IndiGo,24/03/2019,22:20,01:10 22 Mar,2h 50m,non-stop,3897,0,0,0,0,0,0,0,1,0
1,Air India,1/05/2019,05:50,13:15,7h 25m,2 stops,7662,0,0,0,0,0,0,0,1,0
2,Jet Airways,9/06/2019,09:25,04:25 10 Jun,19h,2 stops,13882,0,0,0,0,0,0,0,1,0
3,IndiGo,12/05/2019,18:05,23:30,5h 25m,1 stop,6218,0,0,0,0,0,0,0,1,0
4,IndiGo,01/03/2019,16:50,21:35,4h 45m,1 stop,13302,0,0,0,0,0,0,0,1,0


In [45]:

# trainData['Total_Stops'] = trainData['Total_Stops'].replace(['non-stop','1 stop', '2 stops'],['0','1','2'])
trainData.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/internals/managers.py:1985: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  op = lambda x: operator.eq(x, b)


TypeError: ignored